In [4]:
# =============================================
# 1️⃣ Install / Import Packages
# =============================================
!pip install --pre azure-ai-projects>=2.0.0b1
!pip install azure-identity

import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

In [6]:
from azure.identity import DefaultAzureCredential

cred = DefaultAzureCredential()
token = cred.get_token("https://management.azure.com/.default")
print("Token acquired:", token.token[:20], "...")  # prints first 20 chars

Token acquired: eyJ0eXAiOiJKV1QiLCJh ...


In [7]:


# =============================================
# 2️⃣ Define UKEF Agent (Live Azure Option)
# =============================================
USE_AZURE_AGENT = True  # Set False to use simulated UKEF agent

if USE_AZURE_AGENT:
    myEndpoint = "https://phd-agent-ukef-resource.services.ai.azure.com/api/projects/phd_agent_ukef"
    project_client = AIProjectClient(
        endpoint=myEndpoint,
        credential=DefaultAzureCredential(),
    )
    myAgent = "UKEF1"
    agent = project_client.agents.get(agent_name=myAgent)
    print(f"Retrieved agent: {agent.name}")
    openai_client = project_client.get_openai_client()

    def ask_ukef_agent(question):
        try:
            resp = openai_client.responses.create(
                input=[{"role": "user", "content": question}],
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            )
            answer = resp.output_text
            if not answer.strip():
                return "Simulated fallback: Coverage applies to low-risk projects up to £5M."
            return answer
        except Exception:
            return "Simulated fallback: Coverage applies to low-risk projects up to £5M."
else:
    def ask_ukef_agent(question):
        return "Simulated fallback: Coverage applies to low-risk projects up to £5M."

# =============================================
# 3️⃣ Define PhD Agent (Simulated / Local RAG)
# =============================================
def ask_phd_agent(question):
    folder = "data_phd"
    if not os.path.exists(folder):
        os.makedirs(folder)
        with open(f"{folder}/doc1.txt", "w") as f:
            f.write("This is a sample PhD methodology document covering experiments and analysis.")
    
    docs = [open(f"{folder}/{f}").read() for f in os.listdir(folder) if f.endswith(".txt")]
    context = "\n".join(docs)
    return f"PhD Agent simulated answer:\nContext:\n{context}\nQuestion:\n{question}"

# =============================================
# 4️⃣ Define Finance Agent (Simulated)
# =============================================
def ask_finance_agent(question):
    folder = "data_finance"
    if not os.path.exists(folder):
        os.makedirs(folder)
        with open(f"{folder}/doc1.txt", "w") as f:
            f.write("Finance Agent docs: budgets, risk analysis, and cash flows.")
    
    docs = [open(f"{folder}/{f}").read() for f in os.listdir(folder) if f.endswith(".txt")]
    context = "\n".join(docs)
    return f"Finance Agent simulated answer:\nContext:\n{context}\nQuestion:\n{question}"

# =============================================
# 5️⃣ Orchestrator Agent
# =============================================
def orchestrator_agent(question):
    """
    Orchestrator = agent itself
    Routes queries to child agents and aggregates responses
    """
    q = question.lower()
    if "ukef" in q or "coverage" in q or "mandate" in q:
        return ask_ukef_agent(question)
    elif "phd" in q or "research" in q or "methodology" in q:
        return ask_phd_agent(question)
    elif "finance" in q or "budget" in q:
        return ask_finance_agent(question)
    else:
        # Call all agents and aggregate
        resp1 = ask_ukef_agent(question)
        resp2 = ask_phd_agent(question)
        resp3 = ask_finance_agent(question)
        return f"{resp1}\n---\n{resp2}\n---\n{resp3}"

# =============================================
# 6️⃣ Example Queries / Demo
# =============================================
example_questions = [
    "Explain coverage rules for UKEF projects.",
    "Summarize key points of my PhD methodology.",
    "How does the No Net Cost mandate apply?",
    "What is the maximum project risk we can cover?",
    "Provide budget insights for low-risk projects."
]

for q in example_questions:
    print("Q:", q)
    print("A:", orchestrator_agent(q))
    print("-"*80)


Retrieved agent: UKEF1
Q: Explain coverage rules for UKEF projects.
A: Simulated fallback: Coverage applies to low-risk projects up to £5M.
--------------------------------------------------------------------------------
Q: Summarize key points of my PhD methodology.
A: PhD Agent simulated answer:
Context:
This is a sample PhD methodology document covering experiments and analysis.
Question:
Summarize key points of my PhD methodology.
--------------------------------------------------------------------------------
Q: How does the No Net Cost mandate apply?
A: The 'No Net Cost' mandate requires UKEF to operate so that its long-term financial operations do not create a loss for HM Treasury. (Source: UKEF 'No Net Cost' mandate)
--------------------------------------------------------------------------------
Q: What is the maximum project risk we can cover?
A: Not found in the documents.
---
PhD Agent simulated answer:
Context:
This is a sample PhD methodology document covering experiments

In [8]:
!pip install semantic-kernel azure-identity openai


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.9/910.9 kB 28.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 40.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 40.8 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 47.7 MB/s  0:00:006m0:00:0100:01
  Created wheel for pybars4: filename=pybars4-0.9.13-py3-none-any.whl size=14418 sha256=

In [ ]:
!pip install semantic-kernel azure-identity

In [28]:
import semantic_kernel as sk
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from azure.identity import DefaultAzureCredential

kernel = sk.Kernel()


In [30]:
# --- Define Native Plugins ---

class UKEFPlugin:
    @kernel_function(
        name="UKEFAgent",
        description="Provides policy coverage details and financial limits for UKEF projects."
    )
    def get_coverage(self, query: str) -> str:
        # This is where your CSV-reading logic would live
        return "UKEF policy coverage applies to low-risk projects up to £5M."

class PhDPlugin:
    @kernel_function(
        name="PhDAgent",
        description="Handles academic methodology, research experiments, and thesis evaluation."
    )
    def get_research(self, query: str) -> str:
        return "PhD agent: Specialized in methodology and systematic evaluation."

# --- Register Plugins ---
kernel.add_plugin(UKEFPlugin(), plugin_name="UKEF")
kernel.add_plugin(PhDPlugin(), plugin_name="PhD")

KernelPlugin(name='PhD', description=None, functions={'PhDAgent': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='PhDAgent', plugin_name='PhD', description='Handles academic methodology, research experiments, and thesis evaluation.', parameters=[KernelParameterMetadata(name='query', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x77f5ddf5dd60>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x77f5de781a60>, method=<bound method

In [31]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

async def agentic_orchestrator(query: str):
    # Set behavior to automatically call functions (plugins)
    settings = AzureChatPromptExecutionSettings(
        function_choice_behavior=FunctionChoiceBehavior.Auto()
    )
    
    # We use invoke_prompt. The LLM sees the description of UKEFAgent and PhDAgent
    # and decides which one to call based on the user's intent.
    result = await kernel.invoke_prompt(
        function_name="Orchestrator",
        plugin_name="Main",
        prompt=query,
        settings=settings
    )
    return result
